In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from scipy import stats
import statsmodels as sm
import plotnine as p9
import seaborn as sns
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
df = pd.read_csv('churn_clean.csv',dtype={'locationid':np.int64})
df.head(5)

,CaseOrder,Customer_id,Interaction,UID,City,State,County,Zip,Lat,Lng,...,MonthlyCharge,Bandwidth_GB_Year,Item1,Item2,Item3,Item4,Item5,Item6,Item7,Item8
0,1,K409198,aa90260b-4141-4a24-8e36-b04ce1f4f77b,e885b299883d4f9fb18e39c75155d990,Point Baker,AK,Prince of Wales-Hyder,99927,56.25100,-133.37571,...,172.455519,904.536110,5,5,5,3,4,4,3,4
1,2,S120509,fb76459f-c047-4a9d-8af9-e0f7d4ac2524,f2de8bef964785f41a2959829830fb8a,West Branch,MI,Ogemaw,48661,44.32893,-84.24080,...,242.632554,800.982766,3,4,3,3,4,3,4,4
2,3,K191035,344d114c-3736-4be5-98f7-c72c281e2d35,f1784cfa9f6d92ae816197eb175d3c71,Yamhill,OR,Yamhill,97148,45.35589,-123.24657,...,159.947583,2054.706961,4,4,2,4,4,3,3,3
3,4,D90850,abfa2b40-2d43-4994-b15a-989b8c79e311,dc8a365077241bb5cd5ccd305136b05e,Del Mar,CA,San Diego,92014,32.96687,-117.24798,...,119.956840,2164.579412,4,4,4,2,5,4,3,3
4,5,K662701,68a861fd-0d20-4e51-a587-8a90407ee574,aabb64a116e83fdc4befc1fbab1663f9,Needville,TX,Fort Bend,77461,29.38012,-95.80673,...,149.948316,271.493436,4,4,4,3,4,4,4,5


In [2]:
df.isnull().sum()

CaseOrder               0
Customer_id             0
Interaction             0
UID                     0
City                    0
State                   0
County                  0
Zip                     0
Lat                     0
Lng                     0
Population              0
Area                    0
TimeZone                0
Job                     0
Children                0
Age                     0
Income                  0
Marital                 0
Gender                  0
Churn                   0
Outage_sec_perweek      0
Email                   0
Contacts                0
Yearly_equip_failure    0
Techie                  0
Contract                0
Port_modem              0
Tablet                  0
InternetService         0
Phone                   0
Multiple                0
OnlineSecurity          0
OnlineBackup            0
DeviceProtection        0
TechSupport             0
StreamingTV             0
StreamingMovies         0
PaperlessBilling        0
PaymentMetho

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 50 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   CaseOrder             10000 non-null  int64  
 1   Customer_id           10000 non-null  object 
 2   Interaction           10000 non-null  object 
 3   UID                   10000 non-null  object 
 4   City                  10000 non-null  object 
 5   State                 10000 non-null  object 
 6   County                10000 non-null  object 
 7   Zip                   10000 non-null  int64  
 8   Lat                   10000 non-null  float64
 9   Lng                   10000 non-null  float64
 10  Population            10000 non-null  int64  
 11  Area                  10000 non-null  object 
 12  TimeZone              10000 non-null  object 
 13  Job                   10000 non-null  object 
 14  Children              10000 non-null  int64  
 15  Age                 

In [4]:
df = df.drop(columns=['CaseOrder', 'Customer_id', 'Interaction', 'UID','City', 'State', 'County', 'Zip', 'Lat', 'Lng', 'Population', 'Area', 'TimeZone', 'Email', 'Job', 'Marital', 'Contacts', 'Techie', 'Contract', 'Phone', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'PaperlessBilling', 'PaymentMethod', 'StreamingTV', 'StreamingMovies', 'Port_modem', 'Tablet', 'Item1', 'Item2', 'Item3', 'Item4', 'Item5', 'Item6', 'Item7', 'Item8'])

In [5]:
df = pd.get_dummies(df, drop_first = True)

In [6]:
df.columns

Index(['Children', 'Age', 'Income', 'Outage_sec_perweek',
       'Yearly_equip_failure', 'Tenure', 'MonthlyCharge', 'Bandwidth_GB_Year',
       'Gender_Male', 'Gender_Nonbinary', 'Churn_Yes',
       'InternetService_Fiber Optic', 'InternetService_None', 'Multiple_Yes'],
      dtype='object')

In [7]:
X = df.drop(['Churn_Yes'],1)
y= df['Churn_Yes']

C:\Users\jorda\AppData\Local\Temp\ipykernel_20848\1633312417.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.


In [8]:
feature_names = X.columns
skbest = SelectKBest(score_func = f_classif, k = 'all')
X_features = skbest.fit_transform(X,y)
X_features.shape

(10000, 13)

In [9]:
p_values = pd.DataFrame({'Feature': X.columns, 'p_value':skbest.pvalues_}).sort_values('p_value')
p_values[p_values['p_value']<0.05]
features_to_keep = p_values['Feature'][p_values['p_value']<0.05]
features_to_keep

5                          Tenure
6                   MonthlyCharge
7               Bandwidth_GB_Year
12                   Multiple_Yes
10    InternetService_Fiber Optic
11           InternetService_None
8                     Gender_Male
Name: Feature, dtype: object

In [10]:
df = df.drop(columns = ['Children', 'Age', 'Income', 'Outage_sec_perweek', 'Yearly_equip_failure', 'Gender_Nonbinary'])
df.to_csv(r'C:\Users\jorda\OneDrive\Documents\WGU Stuff\D209\JTompkins_D209PA1.csv')


In [11]:
X = df[features_to_keep]
y = df['Churn_Yes']
X.head(5)

,Tenure,MonthlyCharge,Bandwidth_GB_Year,Multiple_Yes,InternetService_Fiber Optic,InternetService_None,Gender_Male
0,6.795513,172.455519,904.536110,0,1,0,1
1,1.156681,242.632554,800.982766,1,1,0,0
2,15.754144,159.947583,2054.706961,1,0,0,0
3,17.087227,119.956840,2164.579412,0,0,0,1
4,1.670972,149.948316,271.493436,0,1,0,1


In [12]:
y = df['Churn_Yes']
y.head(5)

0    0
1    1
2    0
3    0
4    1
Name: Churn_Yes, dtype: uint8

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = .3, random_state = 13)
X_train.to_csv(r'C:\Users\jorda\OneDrive\Documents\WGU Stuff\D209\JTompkins_X_train.csv')
X_test.to_csv(r'C:\Users\jorda\OneDrive\Documents\WGU Stuff\D209\JTompkins_X_test.csv')
y_train.to_csv(r'C:\Users\jorda\OneDrive\Documents\WGU Stuff\D209\JTompkins_y_train.csv')
y_test.to_csv(r'C:\Users\jorda\OneDrive\Documents\WGU Stuff\D209\JTompkins_y_test.csv')





In [14]:

from sklearn.neighbors import KNeighborsClassifier
steps = [('scaler', StandardScaler()),
        ('knn', KNeighborsClassifier())]
pipeline = Pipeline(steps)
knn_scaled = pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
y_pred_proba = pipeline.predict_proba(X_test)[:,1]

C:\Users\jorda\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.


In [15]:
print("The accuracy of the model is:")
print(knn_scaled.score(X_test, y_test))
print("The AUC of the model is:")
print(roc_auc_score(y_test, y_pred_proba))
print("The confusion matrix for the model is:")
print(confusion_matrix(y_test, y_pred))


C:\Users\jorda\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.


The accuracy of the model is:
0.8506666666666667
The AUC of the model is:
0.8798003894839338
The confusion matrix for the model is:
[[2009  201]
 [ 247  543]]


In [16]:
cm = confusion_matrix(y_test, y_pred)
accuracy_check = (cm[0,0]+cm[1,1])/(y_test.size)
print(accuracy_check)

0.8506666666666667
